# Révision 3 : analyse exploratoire des données

<img src="https://cdn.pixabay.com/photo/2017/07/22/11/46/adventure-2528477_1280.jpg" alt="carte" width="400"/>

*Source : Image par MasterTux de Pixabay*

#### [Pierre-Loic BAYART](https://www.linkedin.com/in/pierreloicbayart/) - Formation développeur d'applications spécialisation data analyst - Webforce3 - Grenoble Ecole de Management

## Analyse statistique simple et nettoyage

- Récupération du **dataset du Titanic** : https://raw.githubusercontent.com/Pierre-Loic/Formation-DA-GEM-2022/main/Datasets/titanic.csv

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
url = "https://raw.githubusercontent.com/Pierre-Loic/Formation-DA-GEM-2022/main/Datasets/titanic.csv"
df = pd.read_csv(url, index_col="PassengerId")
df

In [ ]:
df.info()

In [ ]:
df.describe()

- Analyse des **valeurs manquantes**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.isna().sum()

In [ ]:
sns.heatmap(df.isna())
plt.show()

- Analyse des **corrélations** entre **grandeurs quantitatives**

In [ ]:
# Corrélation de Pearson
sns.heatmap(df.corr(ethod='pearson', numeric_only=True), annot=True)
plt.show()

In [ ]:
# Corrélation de Spearman
sns.heatmap(df.corr(method='spearman', numeric_only=True), annot=True)
plt.show()

In [ ]:
# Corrélation de Kendall
sns.heatmap(df.corr(method='kendall', numeric_only=True), annot=True)
plt.show()

In [ ]:
sns.pairplot(df.select_dtypes(include=["float64", "int64"]), hue="Survived")
plt.show()

- Analyse des **corrélations** entre **grandeurs qualitatives**

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association

In [ ]:
khi2, pval , ddl , contingent_theorique = chi2_contingency(pd.crosstab(df["Survived"], df["Pclass"]))
v_cramer = association(pd.crosstab(df["Survived"], df["Pclass"]), method="cramer")
print(f"P-value pour le lien entre 'Survived' et 'Pclass' :  {pval}")
print(f"V de Cramer pour le lien entre 'Survived' et 'Pclass' :  {v_cramer}")

In [ ]:
khi2, pval , ddl , contingent_theorique = chi2_contingency(pd.crosstab(df["Survived"], df["Sex"]))
v_cramer = association(pd.crosstab(df["Survived"], df["Sex"]), method="cramer")
print(f"P-value pour le lien entre 'Survived' et 'Sex' :  {pval}")
print(f"V de Cramer pour le lien entre 'Survived' et 'Sex' :  {v_cramer}")

In [ ]:
cross = pd.crosstab(df["Survived"], df["Sex"])
cross = cross.div(cross.sum(axis=1), axis=0)
cross.plot(
    kind='bar', stacked=True, colormap='tab10', figsize=(10, 6),
)
plt.show()

In [ ]:
cross = pd.crosstab(df["Survived"], df["Pclass"])
cross = cross.div(cross.sum(axis=1), axis=0)
cross.plot(
    kind='bar', stacked=True, colormap='tab10', figsize=(10, 6),
)
plt.show()

- Analyse des **corrélations** entre une **grandeur qualitative** et une **grandeur quantitative**

In [ ]:
from scipy import stats

In [ ]:
kstat, pval = stats.kruskal(*[group["Fare"].values for name, group in df.groupby("Pclass")])
kstat, pval

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x="Pclass", y="Fare")
#plt.yscale("log")
plt.show()

- Traitement des **valeurs manquantes**

In [ ]:
# Trop de valeurs manquantes
df = df.drop("Cabin", axis=1)
df

In [ ]:
# Très peu de valeurs manquantes : imputation par le mode
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])
df.isna().sum()

In [ ]:
# Beaucoup de valeurs manquantes mais colonne intéressante
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(random_state=0)
num_df = df.select_dtypes(include=["int64", "float64"])
num_df = pd.DataFrame(imp_mean.fit_transform(num_df), columns=num_df.columns)
num_df

In [ ]:
num_df.isna().sum()

## Nettoyage des données aberrantes (outliers)

- **Analyse graphique** des données aberrantes

In [ ]:
sns.histplot(data=df, x="Fare")
plt.show()

In [ ]:
sns.boxplot(data=df, x="Fare")
plt.show()

- **Analyse statistique** des données aberrantes

In [ ]:
#z-score
df[((df["Fare"]-df["Fare"].mean())/df["Fare"].std()).abs()>3]

In [ ]:
# IQR
iqr = df["Fare"].quantile(0.75) - df["Fare"].quantile(0.25)
display(df[df["Fare"] < df["Fare"].quantile(0.25) - 1.5*iqr], df[df["Fare"] > df["Fare"].quantile(0.75) + 1.5*iqr])

## Analyse en composantes principales

- **Standardisation** des donnéees

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(num_df)

- Calcul des **composantes principales**

In [ ]:
pca = PCA()
pca.fit(data_scaled)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_[:3]))

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(x=range(1, num_df.shape[1]+1), height=pca.explained_variance_ratio_*100)
plt.plot(range(1, num_df.shape[1]+1), (pca.explained_variance_ratio_*100).cumsum(), c="red", marker='o')
plt.axhline(y=80, color='black', linestyle='--')
plt.title("Graphique des éboulis")
plt.ylabel("% de variance expliquée")
plt.xlabel("Rang de la composante principale")
plt.show()

- Affichage du **cercle des corrélations**

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
for i in range(0, pca.components_.shape[1]):
    ax.arrow(0,
             0,  # Fléche à l'origine
             pca.components_[0, i],  #0 for PC1
             pca.components_[1, i],  #1 for PC2
             head_width=0.07,
             head_length=0.07, 
             width=0.02,              )

    plt.text(pca.components_[0, i] + 0.05,
             pca.components_[1, i] + 0.05,
             num_df.columns[i])
    
# affichage des lignes horizontales et verticales
plt.plot([-1, 1], [0, 0], color='grey', ls='--')
plt.plot([0, 0], [-1, 1], color='grey', ls='--')
plt.xlabel(f'C_1 ({round(100*pca.explained_variance_ratio_[0], 1)})')
plt.ylabel(f'C_2 ({round(100*pca.explained_variance_ratio_[1], 1)})')
plt.title("Cercle des corrélations (C_1 et C_2)")

# Ajout du cercle unité
an = np.linspace(0, 2 * np.pi, 100)
plt.plot(np.cos(an), np.sin(an))
plt.axis('equal')
plt.show(block=False)

## 🏅 Exercice bilan

- Effectuer un **analyse en composantes principales (ACP)** pour le jeu de données des pingouins : https://raw.githubusercontent.com/Pierre-Loic/Formation-DA-GEM-2022/main/Datasets/penguins.csv

In [ ]:
# A COMPLETER